In [3]:
!pip install -Uqq fastbook
import fastbook
from fastai.vision.all import *
from fastbook import *



<font size="4">In this project, I built a deep learning model for the MNIST dataset using fastai and PyTorch. I modified the data, implemented loss and accuracy functions, and gained experience in deep learning.</font>


In [33]:
path = untar_data(URLs.MNIST)


In [34]:
Path.BASE_PATH = path



<font size="4">This code block reads the images and labels from the training directory and stores them in a format suitable for machine learning algorithms.</font>

In [35]:
train_x = []
train_y = []
labels = os.listdir(path/'training')
labels.sort()
for label in labels:
    for file in (path/'training'/label).ls().sorted():
        image = Image.open(file)
        train_x.append(tensor(image))
        train_y.append(int(label))


<font size=4>This code block splits the dataset into training and validation sets, preprocesses the image and label data, and ensures the label tensors have the correct dimensions for use in the model.<font size>

In [36]:
splitter = RandomSplitter(valid_pct=0.2)
train_idx, valid_idx = splitter(range(len(train_x)))

x_train = [train_x[i] for i in train_idx]
y_train = [train_y[i] for i in train_idx]
x_valid = [train_x[i] for i in valid_idx]
y_valid = [train_y[i] for i in valid_idx]

x_train = (torch.stack(x_train).float()/255).view(-1,28*28)
y_train = tensor(y_train).view(-1,1)
x_valid = (torch.stack(x_valid).float()/255).view(-1,28*28)
y_valid = tensor(y_valid).view(-1,1)

x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

y_train = y_train.squeeze()
y_valid = y_valid.squeeze()

<font size=4>Combining the inputs and labels for the training dataset into tuples, and creating a list of these tuples.<font size>

In [37]:
training = list(zip(x_train, y_train))
validation = list(zip(x_valid, y_valid))


<font size=4> This function calculates the cross entropy loss between the predicted values and the actual labels. First, we apply the log softmax function to the predicted values. Then we calculate the negative log likelihood loss using the actual labels. This loss penalizes the model more when it makes more confident incorrect predictions, and less when it makes less confident incorrect predictions. Lastly we return loss value. <font size>
    

In [38]:
def cross_entropy_loss(predictions, train_y):
    log_softmax = F.log_softmax(predictions, dim=1)
    loss = F.nll_loss(log_softmax, train_y)
    return loss

<font size=4> This function calculates the accuracy of the predicted labels for a batch of inputs 
    by comparing them to the actual target labels. Then function first computes the predicted labels by taking the argmax over the output predictions. It then compares these predicted labels to the actual labels and returns the mean accuracy over the batch. <font size>

In [39]:
def batch_accuracy(preds, targets):
    preds = torch.argmax(preds, dim=1)
    return (preds == targets).float().mean()

<font size=4> Creating PyTorch/FastAI data loaders for our training and validation sets. <font size>

In [40]:
dl = DataLoader(training, batch_size=256)
valid_dl = DataLoader(validation, batch_size=256)
dls = DataLoaders(dl, valid_dl)



(fastai.data.load.DataLoader,
 fastai.data.load.DataLoader,
 fastai.data.core.DataLoaders)

<font size=4> Creating feedforward neural network with ReLU activation functions <font size>

In [42]:
simple_net = nn.Sequential(
    nn.Linear(28*28,100),
    nn.ReLU(),
    nn.Linear(100,50),
    nn.ReLU(),
    nn.Linear(50,10)
   
)

<font size=4> Creating FastAI learner with "our" neural network, loss function and batch accuracy. Using SGD for optimizing.<font size>

In [43]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=cross_entropy_loss, metrics=batch_accuracy,)

<font size=4> And then training with 0.1 lr <font size>
    

In [44]:
learn.fit(20, 0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.670606,0.601179,0.794167,00:01
1,0.374279,0.445963,0.859667,00:01
2,0.309616,0.373854,0.882750,00:01
3,0.272211,0.324112,0.899000,00:01
4,0.243279,0.281811,0.913917,00:01
5,0.218369,0.248300,0.926583,00:01
6,0.197272,0.223467,0.933500,00:01
7,0.179384,0.204190,0.937250,00:01
8,0.164212,0.187359,0.943333,00:01
9,0.151023,0.173701,0.947917,00:01


<font size=4> The goal of this small project was to gain a deeper understanding of key concepts in deep learning such as tensors and data, Stochastic Gradient Descent (SGD), loss functions, neural networks, and batches.
 Mainly using material from fastbook [https://github.com/fastai/fastbook/blob/master/04_mnist_basics.ipynb] <font size>